<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=448d3a30f802702a8740fd6c5c3625909edd808e93520312024bad1b73d6bf54
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.4
    Uninstalling protobuf-6.33.4:
      Successfully uninstalled protobuf-6.33.4


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-20 10:51:58
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.38 C
-------------------
Today PnL: -1.24 L (-0.89%)
Current PnL: -31.16 L (-19.87%)
CY Booked + Current PnL: -16.87 L (-10.76%)
-------------------
Total profit:  1.50 L
Total loss:  -32.66 L
-------------------
Total Booked + Current PnL: 10.11 L (7.79%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.36%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 96.51 L (69.98%)
Deployed:  1.30 C
Current:  1.38 C
CAGR/XIRR %: 3.23%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,-0.27,15.67,4.95,21.40,15198.0,41605.0,307031.0,7.34,61.0,X-LC,6.95,14.0,2.74,2.23,22.99,XY25,NTT,FMCG
17,COALINDIA,484.83,-1.74,3.93,14.71,19.23,23003.0,5920.0,156380.0,21.35,58.0,L-LC,10.45,182.0,0.26,1.13,26.05,XY25,ATH,MINING
77,TTKPRESTIG,770.00,2.22,-22.90,29.83,0.09,23177.0,-23081.0,77696.0,80.36,39.0,M-SC,4.69,253.0,-1.00,0.56,2.22,OX40N,NTT,DURABLES
49,MASFIN,398.61,0.21,-4.72,28.09,22.05,26225.0,-4620.0,93360.0,-17.80,45.0,H-SC,5.07,168.0,-0.18,0.68,35.71,XR,ATH,FINANCE
32,HCLTECH,1908.19,-1.21,10.07,12.50,23.83,33286.0,24364.0,266288.0,14.88,63.0,X-LC,6.38,13.0,0.73,1.93,27.76,X40,ATH,IT


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,2.22,-22.90,29.83,0.09,23177.0,-23081.0,77696.0,80.36,39.0,M-SC,4.69,253.0,-1.00,0.56,2.22,OX40N,NTT,DURABLES
34,HONAUT,58357.33,1.56,-15.90,70.41,43.32,96448.0,-25892.0,136980.0,-27.69,55.0,X-SC,8.07,90.0,-0.27,0.99,5.18,X40N,ATH,ELECTRICAL
11,BATAINDIA,2096.00,0.82,-41.82,132.37,35.20,99100.0,-53804.0,74866.0,-1.81,29.0,X-SC,17.84,93.0,-0.54,0.54,0.82,X40,NTT,FOOTWEAR
85,ZYDUSLIFE,1286.17,0.70,-6.43,44.95,35.63,88144.0,-13481.0,196093.0,-19.84,40.0,H-MC,4.43,119.0,-0.15,1.42,9.17,AR,ATH,PHARMA
33,HINDUNILVR,2922.00,0.56,-5.57,20.38,13.67,49470.0,-14323.0,242740.0,-12.61,62.0,X-LC,2.34,9.0,-0.29,1.76,14.59,XY25,NTT,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,-4.12,-25.98,103.70,50.79,107968.0,-36537.0,104116.0,-4.91,43.0,H-SC,8.80,171.0,-0.34,0.75,21.23,XR,ATH,IT
31,HAVELLS,2069.16,-3.46,-10.56,48.11,32.48,141140.0,-34621.0,293370.0,-15.05,40.0,X-MC,5.08,69.0,-0.25,2.13,0.51,X40,ATH,ELECTRICAL
40,INDUSINDBK,1800.00,-3.03,-32.00,95.44,32.89,50103.0,-24710.0,52497.0,-748.59,59.0,L-MC,10.30,259.0,-0.49,0.38,44.57,XR,NTT,BANKS
18,COFFEEDAY,80.00,-3.00,-43.64,140.02,35.27,89604.0,-49555.0,63994.0,-57.75,42.0,L-SC,5.39,268.0,-0.55,0.46,56.63,XR,NTT,HOTELS
56,RAJOOENG,143.10,-2.94,-35.24,115.58,39.61,76722.0,-36120.0,66380.0,-56.22,47.0,H-SC,29.18,136.0,-0.47,0.48,7.45,AR,ATH,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.0,-0.99,-0.80,21.71,20.74,43756.0,-1622.0,201548.0,31.15,52.0,M-MC,5.10,189.0,-0.04,1.46,9.73,XY25,NTT,FINANCE
84,WIPRO,420.0,-1.48,-0.62,73.34,72.27,123503.0,-1050.0,168398.0,-14.70,33.0,M-LC,5.14,101.0,-0.01,1.22,5.52,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-1.11,-5.29,119.53,107.91,167138.0,-7815.0,139829.0,-22.98,42.0,M-SC,9.90,220.0,-0.05,1.01,4.26,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-2.63,-19.69,24.55,0.02,34415.0,-34377.0,140182.0,113.73,39.0,M-SC,9.07,240.0,-1.00,1.02,21.93,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,-1.54,-21.06,67.25,32.03,69988.0,-27763.0,104072.0,-14.53,51.0,H-SC,6.36,157.0,-0.40,0.75,16.57,OX40N,NTT,IT
77,TTKPRESTIG,770.00,2.22,-22.90,29.83,0.09,23177.0,-23081.0,77696.0,80.36,39.0,M-SC,4.69,253.0,-1.00,0.56,2.22,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-2.33,-24.14,49.58,13.47,101430.0,-65088.0,204579.0,-68.90,42.0,H-SC,4.23,159.0,-0.64,1.48,4.02,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,COALINDIA,484.83,-1.74,3.93,14.71,19.23,23003.0,5920.0,156380.0,21.35,58.0,L-LC,10.45,182.0,0.26,1.13,26.05,XY25,ATH,MINING


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,-1.48,-0.62,73.34,72.27,123503.0,-1050.0,168398.0,-14.70,33.0,M-LC,5.14,101.0,-0.01,1.22,5.52,XR,NTT,IT
25,FINCABLES,1641.55,-1.11,-5.29,119.53,107.91,167138.0,-7815.0,139829.0,-22.98,42.0,M-SC,9.90,220.0,-0.05,1.01,4.26,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-0.73,-6.39,120.84,106.72,139510.0,-7886.0,115450.0,-53.68,47.0,H-SC,8.56,138.0,-0.06,0.84,15.49,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,0.70,-6.43,44.95,35.63,88144.0,-13481.0,196093.0,-19.84,40.0,H-MC,4.43,119.0,-0.15,1.42,9.17,AR,ATH,PHARMA
37,IEX,219.00,-1.99,-8.88,63.10,48.63,114971.0,-17750.0,182204.0,-39.32,42.0,H-SC,13.12,137.0,-0.15,1.32,1.71,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-0.26,-31.46,129.35,57.19,289128.0,-102603.0,223524.0,-67.59,16.0,X-MC,12.32,58.0,-0.35,1.62,0.00,XY24,NTT,FMCG
43,ITC,452.00,0.01,-18.32,35.63,10.78,69936.0,-44034.0,196284.0,-51.37,19.0,X-LC,11.71,1.0,-0.63,1.42,1.23,X40,NTT,FMCG
58,RELAXO,1176.00,-0.03,-51.25,212.43,52.31,150322.0,-74397.0,70763.0,-48.12,23.0,X-SC,8.77,92.0,-0.49,0.51,0.00,X40N,NTT,FOOTWEAR
53,PGHH,17907.65,-1.18,-9.94,48.99,34.19,94210.0,-21216.0,192304.0,-36.67,23.0,X-MC,5.11,60.0,-0.23,1.39,0.00,X40,ATH,FMCG
59,RELIANCE,1952.00,-0.50,-9.72,38.78,25.30,70362.0,-19525.0,181438.0,-12.63,25.0,X-LC,6.36,26.0,-0.28,1.32,21.53,XY25,BTT,REFINERIES


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4311.58,-0.84,-13.75,37.45,18.55,122176.0,-51999.0,326238.0,-25.57,46.0,X-LC,1.26,3.0,-0.43,2.37,10.99,X40,ATH,IT
45,JIOFIN,387.00,-1.60,-12.75,42.67,24.48,99538.0,-34090.0,233275.0,-14.25,28.0,X-LC,2.09,37.0,-0.34,1.69,35.21,XY24,BTT,FINANCE
20,DABUR,735.00,-0.19,0.05,43.62,43.70,108264.0,131.0,248199.0,-8.14,52.0,X-MC,2.10,73.0,0.00,1.80,15.11,XY24,BTT,FMCG
78,UNITDSPR,1644.00,0.15,-2.53,23.91,20.77,54565.0,-5934.0,228210.0,-11.08,35.0,X-MC,2.21,70.0,-0.11,1.65,4.21,X40N,NTT,BREWERIES
33,HINDUNILVR,2922.00,0.56,-5.57,20.38,13.67,49470.0,-14323.0,242740.0,-12.61,62.0,X-LC,2.34,9.0,-0.29,1.76,14.59,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-0.26,-31.46,129.35,57.19,289128.0,-102603.0,223524.0,-67.59,16.0,X-MC,12.32,58.0,-0.35,1.62,0.0,XY24,NTT,FMCG
8,BAJAJHFL,181.50,-0.88,-22.43,100.44,55.47,189536.0,-54580.0,188706.0,-31.26,25.0,X-MC,16.44,63.0,-0.29,1.37,0.0,X40N,ATH,FINANCE
9,BAJAJHLDNG,14451.52,-0.24,-5.02,36.05,29.22,65097.0,-9551.0,180574.0,-7.33,33.0,X-LC,12.20,29.0,-0.15,1.31,0.0,X40,ATH,FINANCE
53,PGHH,17907.65,-1.18,-9.94,48.99,34.19,94210.0,-21216.0,192304.0,-36.67,23.0,X-MC,5.11,60.0,-0.23,1.39,0.0,X40,ATH,FMCG
58,RELAXO,1176.00,-0.03,-51.25,212.43,52.31,150322.0,-74397.0,70763.0,-48.12,23.0,X-SC,8.77,92.0,-0.49,0.51,0.0,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-1.25,-31.72,109.21,42.84,48473.0,-20621.0,44385.0,-55.37,37.0,X-SC,23.35,83.0,-0.43,0.32,0.42,XY24,NTT,MISC
58,RELAXO,1176.00,-0.03,-51.25,212.43,52.31,150322.0,-74397.0,70763.0,-48.12,23.0,X-SC,8.77,92.0,-0.49,0.51,0.00,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.82,-41.82,132.37,35.20,99100.0,-53804.0,74866.0,-1.81,29.0,X-SC,17.84,93.0,-0.54,0.54,0.82,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-2.05,-8.86,36.14,24.08,41814.0,-11251.0,115699.0,-16.24,32.0,X-SC,6.37,91.0,-0.27,0.84,9.01,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,1.56,-15.90,70.41,43.32,96448.0,-25892.0,136980.0,-27.69,55.0,X-SC,8.07,90.0,-0.27,0.99,5.18,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,0.01,-18.32,35.63,10.78,69936.0,-44034.0,196284.0,-51.37,19.0,X-LC,11.71,1.0,-0.63,1.42,1.23,X40,NTT,FMCG
75,TMPV,600.00,-0.86,-30.10,75.93,22.98,119898.0,-67992.0,157906.0,-27.96,33.0,X-LC,3.57,2.0,-0.57,1.15,0.00,XY24,NTT,AUTO
73,TCS,4311.58,-0.84,-13.75,37.45,18.55,122176.0,-51999.0,326238.0,-25.57,46.0,X-LC,1.26,3.0,-0.43,2.37,10.99,X40,ATH,IT
81,VBL,671.64,0.00,0.19,35.41,35.67,112054.0,606.0,316448.0,-11.70,55.0,X-LC,3.00,5.0,0.01,2.29,14.07,X40N,ATH,FMCG
41,INFY,1972.00,-0.61,13.75,18.01,34.24,64102.0,43035.0,355923.0,-11.34,61.0,X-LC,7.42,6.0,0.67,2.58,23.18,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-3.03,-32.00,95.44,32.89,50103.0,-24710.0,52497.0,-748.59,59.0,L-MC,10.30,259.0,-0.49,0.38,44.57,XR,NTT,BANKS
17,COALINDIA,484.83,-1.74,3.93,14.71,19.23,23003.0,5920.0,156380.0,21.35,58.0,L-LC,10.45,182.0,0.26,1.13,26.05,XY25,ATH,MINING
36,ICICIPRULI,790.00,-0.19,9.17,19.75,30.73,42475.0,18057.0,215062.0,-16.33,49.0,X-MC,7.43,79.0,0.43,1.56,23.10,X40,ATH,INSURANCE
5,ASIANPAINT,3460.25,-1.51,-5.19,27.57,20.95,65807.0,-13077.0,238691.0,-6.11,40.0,X-LC,12.77,36.0,-0.20,1.73,28.59,X40,ATH,PAINTS
32,HCLTECH,1908.19,-1.21,10.07,12.50,23.83,33286.0,24364.0,266288.0,14.88,63.0,X-LC,6.38,13.0,0.73,1.93,27.76,X40,ATH,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-0.78,-28.41,57.71,12.90,90268.0,-62080.0,156416.0,130.77,59.0,H-SC,7.70,173.0,-0.69,1.13,30.06,OX40N,NTT,FINANCE
17,COALINDIA,484.83,-1.74,3.93,14.71,19.23,23003.0,5920.0,156380.0,21.35,58.0,L-LC,10.45,182.0,0.26,1.13,26.05,XY25,ATH,MINING
40,INDUSINDBK,1800.00,-3.03,-32.00,95.44,32.89,50103.0,-24710.0,52497.0,-748.59,59.0,L-MC,10.30,259.0,-0.49,0.38,44.57,XR,NTT,BANKS
62,SATIN,274.00,-0.87,-16.79,82.47,51.83,189594.0,-46389.0,229895.0,-25.10,59.0,H-SC,3.55,148.0,-0.24,1.67,11.99,XY24,NTT,FINANCE
57,RECLTD,446.00,-0.99,-0.80,21.71,20.74,43756.0,-1622.0,201548.0,31.15,52.0,M-MC,5.10,189.0,-0.04,1.46,9.73,XY25,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.61
1,25,44.82
2,50,76.75


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.44
MC    30.72
LC    26.80
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.17
X40      24.14
X40N     12.92
XY25     10.98
XR        8.78
AR        8.37
OX40N     7.65
SR        0.95
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.33
X-LC    23.39
H-SC    22.71
M-SC    11.04
X-SC     7.91
H-MC     4.55
M-MC     1.46
M-LC     1.22
L-LC     1.13
H-LC     1.06
L-SC     0.78
L-MC     0.38
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.42,-9.51,45.44
IT,12.80,-21.00,81.45
FINANCE,11.09,-16.57,65.49
MISC,7.03,-33.91,88.69
ELECTRICAL,5.83,-16.74,59.49
PAINTS,5.74,-16.69,34.03
INSURANCE,4.74,-3.86,39.41
PHARMA,3.96,-7.63,41.51
BANKS,2.79,-39.00,123.22


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3361450.0,21
AR,1416059.0,10
XR,1281756.0,12
X40,1180035.0,15
X40N,957429.0,9
OX40N,737004.0,10
XY25,427943.0,7
SR,289302.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3662480.0,24
X-MC,1677488.0,16
M-SC,1535546.0,15
X-LC,987730.0,13
X-SC,825739.0,8
H-MC,450918.0,3
L-SC,183133.0,2
M-LC,123503.0,1
H-LC,87579.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1282813.0      6
           AR         950822.0      5
M-SC       XY24       876726.0      6
H-SC       XR         825683.0      7
X-MC       X40        563060.0      7
           XY24       464991.0      3
           X40N       423283.0      4
X-LC       X40        420404.0      6
M-SC       OX40N      329615.0      5
X-SC       X40N       321286.0      3
H-SC       OX40N      313860.0      4
X-SC       XY24       307882.0      3
H-SC       SR         289302.0      2
H-MC       AR         241316.0      2
X-MC       XY25       226154.0      2
X-LC       XY24       219436.0      2
           X40N       212860.0      2
H-MC       XY24       209602.0      1
X-SC       X40        196571.0      2
M-SC       XR         192863.0      2
           AR         136342.0      2
X-LC       XY25       135030.0      3
M-LC       XR         123503.0      1
L-SC       OX40N       93529.0      1
           XR          89604.0      1
H-LC       AR          87579.0      1
L-MC       XR          50103.0      1
M-MC       XY25        43756.0      1
L-LC       XY25        23003.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 49.0 seconds
